##   Import Module$

In [289]:
import pickle
import random

## Utility Function$

In [290]:

# $ample True or False for a given success probability
def with_probability(probability):
    # Generate a random number between 0 and 1
    random_value = random.uniform(0, 1)
    
    # Check if the random number is less than the given probability
    return random_value < probability


# $ample a category from a given categorie$ with probability given a$ a dictionary
def sample_from_probability(probabilities):
    # Check if the probabilities sum to 1
    if abs(sum(probabilities.values()) - 1.0) > 1e-10:
        raise ValueError("Probabilities must sum to 1")

    # Sample from the given probability distribution
    outcome = random.choices(list(probabilities.keys()), weights=list(probabilities.values()), k=1)[0]
    return outcome

#Load the error model for generating error in the sentence
with open('models/bma_27dec.pickle','rb') as f:
    bma = pickle.load(f)

In [75]:
bma.edit_dict

defaultdict(int,
            {(None, 'ि'): 0.06163917824074074,
             ('फ', 'प'): 0.11700581395348837,
             (None, 'य'): 0.043243634259259256,
             (None, 'ा'): 0.1424045138888889,
             ('ि', 'ी'): 0.7614195183776933,
             ('व', 'म'): 0.00959983831851253,
             ('ध', 'म'): 0.09599783666846945,
             ('अ', 'आ'): 0.044686116630067274,
             ('ी', 'ि'): 0.296859385351284,
             (None, '्'): 0.08172019675925926,
             ('द', 'ध'): 0.08529155787641428,
             ('ः', None): 0.8502824858757062,
             ('ै', 'ा'): 0.5024773091297384,
             ('म', None): 0.17509884974838247,
             (None, 'े'): 0.04787326388888889,
             ('क', 'त'): 0.06099039172209904,
             (None, 'ी'): 0.08294994212962963,
             ('व', 'ब'): 0.8117168552950688,
             ('्', None): 0.7254199506408725,
             ('ा', 'ै'): 0.0734890251708687,
             ('ज', 'म'): 0.19491060655917516,
             ('

In [291]:
# Probability dict $elector

dict_selector = {}
for k1 in bma.edit_dict:
    temp_dict = {}
    if k1[0] in dict_selector:        
        continue
    for k2 in bma.edit_dict:
        if k2[0] == k1[0]:
            temp_dict[k2] = bma.edit_dict[k2]
    dict_selector[k1[0]] = temp_dict
    

In [80]:
dict_selector.keys()

dict_keys([None, 'फ', 'ि', 'व', 'ध', 'अ', 'ी', 'द', 'ः', 'ै', 'म', 'क', '्', 'ा', 'ज', 'प', 'न', 'भ', 'ल', 'ू', 'त', 'े', 'ऊ', 'ब', 'ण', 'स', 'ह', 'ु', 'ए', 'ङ', 'य', 'ो', 'ँ', 'च', 'आ', 'ग', 'र', 'इ', 'ई', 'श', 'ष', 'छ', 'ं', 'झ', 'ट', 'उ', 'घ', 'ौ', 'थ', 'ख', 'ृ', 'ड', 'ठ', 'ओ', 'ढ', 'ञ', 'ऐ', 'औ'])

## $ample Generation

In [362]:

# $ample Generation of error
text = 'नेपालीले धेरै काम गर्छन'
tokens = text.split()
orig_tokens = tokens.copy()
print("Probability of error in each character", 1-bma.error_rate)

Probability of error in each character 0.010000000000000009


In [380]:

for i in range(len(tokens)):
    
    # List that contains True if the character is to be errored 
    result = [with_probability(1-bma.error_rate) for i in range(len(tokens[i]))]
    print(result)
    
    # List with replacement character if the character is to be errored
    li = [ch if (b!= True or ch not in dict_selector) else sample_from_probability(dict_selector[ch])[1] for ch,b in zip(tokens[i],result)]
    print(li)
    
    # Create a final token by joining
    tokens[i] = ''.join(['' if ch is None else ch for ch in li])
    


[False, False, False, False, False, False, False, False]
['न', 'े', 'प', 'ा', 'ल', 'ी', 'ल', 'े']
[False, False, True, False]
['ध', 'े', None, 'ै']
[False, False, False]
['क', 'ा', 'म']
[False, False, False, False]
['ग', 'र', 'छ', 'न']


In [381]:
tokens,orig_tokens

(['नेपालीले', 'धेै', 'काम', 'गरछन'], ['नेपालीले', 'धेरै', 'काम', 'गर्छन'])

## Passing through the noisy channel

In [387]:
import regex as re
def words(text): 
    text = re.sub(r'[\u0964]', r'\u0020\u0964\u0020', text)
    return re.findall(r'[\u0900-\u097F]+', text.lower())

In [382]:
with open('data/wikipedia.dev.ne-en.ne','r',encoding = 'utf-8') as f:
    sentences = f.read().split('\n')

In [385]:
sentences

["पुटिनको आफ्नै 'नाइट उल्भ्स' नामको बाइक राइडर ग्याङ पनि छ ।",
 "पुटिनको तर्क छ, अमेरिका र युरोपको आडमा कोसोभोले 'आत्मनिर्णयको अधिकार' उपयोग गर्न पाउँछ भने रुसको समर्थन पाएको क्रिमियाले नपाउने कारण के छ?",
 'पुटिनको विरोधमा गरिने सबै र् याली र सभाहरूमा यो गीत आधिकारिकजस्तै बनेको थियो ।',
 "पुटिनको शब्दमा ' घुँडा टेकेर बसेको रुस अब बिस्तारै आफ्नो खुट्टामा उभिएको छ ।",
 "पुटिन राष्ट्रपति भएको दुई वर्ष हुँदा लेखिएको गीत हो 'अ म्यान लाइक पुटिन' ।",
 'पुटिनलाई गीतमा देशलाई सही दिशामा डोहोर् याउने असल नेता, उदाहरणीय व्यक्तित्व र अनुकरणीय पतिका रूपमा देखाइएको छ ।',
 'पुटिनले आफ्ना सुरुआती दुई कार्यकालमा तेल र ग्याँस बेचेर रुसको अर्थतन्त्र उकासेका थिए ।',
 'पुरानो गुप्तचरलाई घर छिर्ने अनुमति दिँदा के हुन्छ भनेर उनैले सोच्नुपथ्र्यो ।',
 "पुँजीवादको सूक्ष्मतासँग अनभिज्ञ रुसीहरूलाई विश्व बैंक र विशेषगरी अन्तर्राष्ट्रिय मुद्रा कोषका 'कन्सल्ट्यान्ट' का सुझावमा चल्नुपर्ने बाध्यता थियो ।",
 'पूरै विश्व उनको विरुद्धमा लाग्दा पनि पुटिनले जसरी क्रिमियालाई आफ्नो बनाएरै छाडे, उनले आफ्नो प्रेमी पनि त्यस्तै

In [389]:
sentences_tokens = [words(s) for s in sentences]
sentences_tokens

[['पुटिनको',
  'आफ्नै',
  'नाइट',
  'उल्भ्स',
  'नामको',
  'बाइक',
  'राइडर',
  'ग्याङ',
  'पनि',
  'छ',
  '।'],
 ['पुटिनको',
  'तर्क',
  'छ',
  'अमेरिका',
  'र',
  'युरोपको',
  'आडमा',
  'कोसोभोले',
  'आत्मनिर्णयको',
  'अधिकार',
  'उपयोग',
  'गर्न',
  'पाउँछ',
  'भने',
  'रुसको',
  'समर्थन',
  'पाएको',
  'क्रिमियाले',
  'नपाउने',
  'कारण',
  'के',
  'छ'],
 ['पुटिनको',
  'विरोधमा',
  'गरिने',
  'सबै',
  'र्',
  'याली',
  'र',
  'सभाहरूमा',
  'यो',
  'गीत',
  'आधिकारिकजस्तै',
  'बनेको',
  'थियो',
  '।'],
 ['पुटिनको',
  'शब्दमा',
  'घुँडा',
  'टेकेर',
  'बसेको',
  'रुस',
  'अब',
  'बिस्तारै',
  'आफ्नो',
  'खुट्टामा',
  'उभिएको',
  'छ',
  '।'],
 ['पुटिन',
  'राष्ट्रपति',
  'भएको',
  'दुई',
  'वर्ष',
  'हुँदा',
  'लेखिएको',
  'गीत',
  'हो',
  'अ',
  'म्यान',
  'लाइक',
  'पुटिन',
  '।'],
 ['पुटिनलाई',
  'गीतमा',
  'देशलाई',
  'सही',
  'दिशामा',
  'डोहोर्',
  'याउने',
  'असल',
  'नेता',
  'उदाहरणीय',
  'व्यक्तित्व',
  'र',
  'अनुकरणीय',
  'पतिका',
  'रूपमा',
  'देखाइएको',
  'छ',
  '।'],
 ['प

In [400]:
#error_rate = bma.error_rate
error_rate = 0.9

def make_error(tokens):
    
    for i in range(len(tokens)):

        # List that contains True if the character is to be errored 
        result = [with_probability(1-error_rate) for i in range(len(tokens[i]))]

        # List with replacement character if the character is to be errored
        li = [ch if (b!= True or ch not in dict_selector) else sample_from_probability(dict_selector[ch])[1] for ch,b in zip(tokens[i],result)]

        # Create a final token by joining
        tokens[i] = ''.join(['' if ch is None else ch for ch in li])
        
    return tokens

In [401]:
correct_incorrect = []
for t in sentences_tokens:
    correct_incorrect.append((' '.join(t),' '.join(make_error(t.copy()))))
    
    

In [403]:
len(correct_incorrect),correct_incorrect

(2560,
 [('पुटिनको आफ्नै नाइट उल्भ्स नामको बाइक राइडर ग्याङ पनि छ ।',
   'पुटिनको आफ्नै नाइट कल्भ्ज नामको बाइक राइमर ग्याङ डनि छ ।'),
  ('पुटिनको तर्क छ अमेरिका र युरोपको आडमा कोसोभोले आत्मनिर्णयको अधिकार उपयोग गर्न पाउँछ भने रुसको समर्थन पाएको क्रिमियाले नपाउने कारण के छ',
   'पुटिनको तर्क छ अमेरिका ज युसोपको आडमा कोसोभोले आम्मनिर्कयको अविकार उपयोग जर्क पाउँछ भन रुसको हमर्थन पाअको क्रीमियाले नकाउे मारण कण छ'),
  ('पुटिनको विरोधमा गरिने सबै र् याली र सभाहरूमा यो गीत आधिकारिकजस्तै बनेको थियो ।',
   'पुटिनकस विरोधमा गरिनो सबै रा याली र सभाहरूमा यो गत आधिनोरिकजउ्कै बनाको थियो ।'),
  ('पुटिनको शब्दमा घुँडा टेकेर बसेको रुस अब बिस्तारै आफ्नो खुट्टामा उभिएको छ ।',
   'पूटिनको शव्दम घुँडा टेकेर बस्को रुस ब बिस्तारै भफ्नो खुट्टामे उभिएको छ ।'),
  ('पुटिन राष्ट्रपति भएको दुई वर्ष हुँदा लेखिएको गीत हो अ म्यान लाइक पुटिन ।',
   'पुचिन राष्ट्रपति भवके दूई वर्ष हुँदा लेखिएकै गीय हो अ म्यान लाइक पुटीन ।'),
  ('पुटिनलाई गीतमा देशलाई सही दिशामा डोहोर् याउने असल नेता उदाहरणीय व्यक्तित्व र अनुकरणीय पतिका

## Passing through the noisy model with variable error rate

In [431]:
total = sum(bma.count_dict.values())
count_probab = {k:(bma.count_dict[k]/total) for k in bma.count_dict}
count_probab['n'] = count_probab[None]

In [415]:
sum(count_probab.values())

0.9999999999999999

In [432]:
sorted(count_probab.items(),key = lambda x:x[1],reverse = True)

[(None, 0.12892972658392185),
 ('n', 0.12892972658392185),
 ('ा', 0.09033474723282336),
 ('ै', 0.08734280161012933),
 ('ी', 0.08306099913076892),
 ('क', 0.06309386576535238),
 ('ि', 0.05518964533133373),
 ('ब', 0.05017198091422219),
 ('न', 0.038807623865429595),
 ('व', 0.036918072172294286),
 ('ु', 0.03567111727904556),
 ('अ', 0.02675683540195409),
 ('स', 0.02346737398947223),
 ('म', 0.020757089083129083),
 ('श', 0.017432186918258704),
 ('ल', 0.015166776719529347),
 ('र', 0.014509257497584433),
 ('ू', 0.014079305212028964),
 ('त', 0.013999097193465471),
 ('्', 0.01171503396716321),
 ('ई', 0.010788911148168458),
 ('ह', 0.010687252147896125),
 ('ो', 0.010188283660320907),
 ('े', 0.00960444389728897),
 ('प', 0.009282679171656351),
 ('ज', 0.008502982619108907),
 ('ग', 0.008237176976195006),
 ('ट', 0.007549812910133444),
 ('द', 0.00750131503844389),
 ('य', 0.0073548887719965825),
 ('च', 0.006921205880926533),
 ('भ', 0.005440155491637848),
 ('छ', 0.005182743711131753),
 ('ष', 0.0050167317657

In [430]:
dict_selector['n'] = dict_selector[None]
dict_selector['n']

{(None, 'ि'): 0.06163917824074074,
 (None, 'य'): 0.043243634259259256,
 (None, 'ा'): 0.1424045138888889,
 (None, '्'): 0.08172019675925926,
 (None, 'े'): 0.04787326388888889,
 (None, 'ी'): 0.08294994212962963,
 (None, 'अ'): 0.02664207175925926,
 (None, 'न'): 0.06960358796296297,
 (None, 'ै'): 0.04216579861111111,
 (None, 'स'): 0.017549189814814816,
 (None, 'ं'): 0.019943576388888887,
 (None, 'व'): 0.009946469907407407,
 (None, 'क'): 0.018677662037037038,
 (None, 'प'): 0.005092592592592593,
 (None, 'त'): 0.022432002314814816,
 (None, 'ष'): 0.0009186921296296297,
 (None, 'ण'): 0.022800925925925926,
 (None, 'म'): 0.03448350694444444,
 (None, 'ु'): 0.016109664351851852,
 (None, 'र'): 0.03994502314814815,
 (None, 'ह'): 0.003486689814814815,
 (None, 'ल'): 0.02093460648148148,
 (None, 'ध'): 0.010243055555555556,
 (None, 'आ'): 0.019220196759259258,
 (None, 'ट'): 0.007248263888888889,
 (None, 'द'): 0.005577256944444445,
 (None, 'इ'): 0.0023582175925925927,
 (None, 'ँ'): 0.041652199074074074,
 (

In [468]:


def make_error2(tokens):
    
    for i in range(len(tokens)):
        
        # Add None token between characters to model the error that can happen due to addition of string
        temp = ''
        for j,t in enumerate(tokens[i]):
            temp+=t

            if j == len(tokens[i])-1:
                continue
                
            # Don't add 'n' or None token when previo$ and ju$t after token are character modifier
            if t not in ['े','ै','ो','ौ','्','ॎ','ि','ी','ु','ू','ृ','ा'] and tokens[i][j+1] not in ['े','ै','ो','ौ','्','ॎ','ि','ी','ु','ू','ृ','ा']:
                temp+='n'
    
        
        # List that contains True if the character is to be errored 
        result = [with_probability(count_probab[t]) if t in dict_selector else False for t in temp]

        # List with replacement character if the character is to be errored
        li = [ch if (b!= True or ch not in dict_selector) else sample_from_probability(dict_selector[ch])[1] for ch,b in zip(temp,result)]

        # Create a final token by joining
        tokens[i] = ''.join(['' if ch is None or ch == 'n' else ch for ch in li])
        
    return tokens

In [469]:
correct_incorrect2 = []
for t in sentences_tokens:
    correct_incorrect2.append((' '.join(t),' '.join(make_error2(t.copy()))))
    
    

In [470]:
correct_incorrect2

[('पुटिनको आफ्नै नाइट उल्भ्स नामको बाइक राइडर ग्याङ पनि छ ।',
  'पुटिनको आकफ्नो नाइट उल्भ्स नामको बोइक राइडँर ग्याङ पनि छ ।'),
 ('पुटिनको तर्क छ अमेरिका र युरोपको आडमा कोसोभोले आत्मनिर्णयको अधिकार उपयोग गर्न पाउँछ भने रुसको समर्थन पाएको क्रिमियाले नपाउने कारण के छ',
  'पुटिछको तैर्क छ अमेरिका र युरोपको आडमा कोसोभोले आत्मननिर्णयको अाधिकार उपूयोग गर्न पाउँछ भने रुसको समर्थन पाएको क्रिमियाले नपाउरने कारण के छ'),
 ('पुटिनको विरोधमा गरिने सबै र् याली र सभाहरूमा यो गीत आधिकारिकजस्तै बनेको थियो ।',
  'पुटिनको विरोधमा गङरिने सबै र् योली र सभाहरूमा यो गीत आआधिारिकजस्तै बनेको थियो ।'),
 ('पुटिनको शब्दमा घुँडा टेकेर बसेको रुस अब बिस्तारै आफ्नो खुट्टामा उभिएको छ ।',
  'पुटिनको सब्दमा घुँ्डा टेकेर बसेको रुस अव बिस्तारै भफ्नो खुट्टामा उभएको छ ।'),
 ('पुटिन राष्ट्रपति भएको दुई वर्ष हुँदा लेखिएको गीत हो अ म्यान लाइक पुटिन ।',
  'पुटिन राष्ट्रपति भीएको दुई वर्ष हुँदा लेखिएको गीत हो अ म्यान लाइक पुटिन ।'),
 ('पुटिनलाई गीतमा देशलाई सही दिशामा डोहोर् याउने असल नेता उदाहरणीय व्यक्तित्व र अनुकरणीय पतिका रूप

In [428]:
for t in 'कमिला':
    print(t)

क
म
ि
ल
ा


In [471]:
correct_incorrect2 = []
for t in sentences_tokens:
    correct_incorrect2.append((' '.join(t),' '.join(make_error2(t.copy()))))
    

In [472]:
correct_incorrect2

[('पुटिनको आफ्नै नाइट उल्भ्स नामको बाइक राइडर ग्याङ पनि छ ।',
  'पुटिनको आफ्नै नाइिट उल्भ्स नामको लाइक राइडर ग्याङ पनि छ ।'),
 ('पुटिनको तर्क छ अमेरिका र युरोपको आडमा कोसोभोले आत्मनिर्णयको अधिकार उपयोग गर्न पाउँछ भने रुसको समर्थन पाएको क्रिमियाले नपाउने कारण के छ',
  'पुटिनको तर्क छ अमेरिका र युरोपको आडम कोसोभोले आत्मनिर्णयको अधिकार उपयोग गर्न पाउँछ भने रुसको समर्थ्न पाएको क्रीमियाले नपाउने कावण के छ'),
 ('पुटिनको विरोधमा गरिने सबै र् याली र सभाहरूमा यो गीत आधिकारिकजस्तै बनेको थियो ।',
  'पुटिनको विरोधमा गरिने सबै र् याली र भाहरूमा यो गीत आधिकारीकजस्तै बनेको थियो ।'),
 ('पुटिनको शब्दमा घुँडा टेकेर बसेको रुस अब बिस्तारै आफ्नो खुट्टामा उभिएको छ ।',
  'पूटिनको शब्दमा घुँीडा टेकेर बसेको रुस अब बिस्तारै आफ्नो खुट्टामा उभिएको छ ।'),
 ('पुटिन राष्ट्रपति भएको दुई वर्ष हुँदा लेखिएको गीत हो अ म्यान लाइक पुटिन ।',
  'पुटिन राष्ट्रपति भएको दुई वर्ष हुँदा लेखिएको गीत हो अ म्यान लाइक पुटिन ।'),
 ('पुटिनलाई गीतमा देशलाई सही दिशामा डोहोर् याउने असल नेता उदाहरणीय व्यक्तित्व र अनुकरणीय पतिका रूपमा देखाइ